# 🏥 NeuroFHIR Real-Data Quickstart

Welcome to **NeuroFHIR**, the library for Geometric Deep Learning on clinical data. 

This notebook demonstrates how to:
1.  **Load Patient Data**: Process local FHIR JSON files.
2.  **Build Temporal Graphs**: Convert full patient history into graph snapshots.
3.  **Visualizes the Graph**: Plot the connectivity of healthcare events.
4.  **Hyperbolic Embeddings**: Embed concepts in hyperbolic space.
5.  **Causal Discovery**: Mine potential causal edges.

## Setup

In [ ]:
!pip install neurofhir polars torch networkx matplotlib

import os
import sys
import json
import glob
import networkx as nx
import matplotlib.pyplot as plt

# Ensure we can import local modules if finding from repo root
try:
    import neurofhir
except ImportError:
    # If running from repo, add parent dir
    sys.path.append(os.path.abspath('..'))

## 1. Load a Patient File
We assume you have FHIR data in the `../data` folder. If not, use the `download_sample_data.py` script provided in the repo, or upload your own bundles.

In [ ]:
data_dir = os.path.join("..", "data", "fhir")
# Look for downloaded data or just in ../data depending on extraction
if not os.path.exists(data_dir):
    data_dir = os.path.join("..", "data")

files = glob.glob(os.path.join(data_dir, "*.json"))

if not files:
    print("No files found! Please upload FHIR bundles to the 'data' directory.")
else:
    patient_file = files[0]
    print(f"Selected patient file: {patient_file}")
    
    with open(patient_file, 'r', encoding='utf-8') as f:
        bundle = json.load(f)
        resources = [e['resource'] for e in bundle.get('entry', []) if 'resource' in e]
        print(f"Loaded {len(resources)} FHIR resources.")

## 2. Generate Temporal Graph
We use `FHIRTemporalGraphBuilder` to slice this history into daily snapshots.

In [ ]:
from neurofhir.temporal_builder import FHIRTemporalGraphBuilder

# 1 day windows
builder = FHIRTemporalGraphBuilder(time_window="1d")

snapshots = list(builder.build_snapshots(resources))

print(f"\nGenerated {len(snapshots)} snapshots from patient history.")

## 3. Visualizes the Graph
We will pick the busiest day (snapshot with the most interactions) and plot it using NetworkX.

In [ ]:
# Find snapshot with max edges
if snapshots:
    # Helper to count edges in dict or PyG object
    def count_edges(snap):
        if isinstance(snap, dict):
            # Use 'edge_index_dict' key for raw builder output
            if 'edge_index_dict' in snap:
                return sum(len(v[0]) for v in snap['edge_index_dict'].values())
            # Fallback for 'edges' key if serialized
            if 'edges' in snap:
                return sum(len(v[0]) for v in snap['edges'].values())
            return 0
        return snap.num_edges

    max_snap = max(snapshots, key=count_edges)
    idx = snapshots.index(max_snap)
    
    print(f"Visualizing Snapshot {idx} (Timestamp: {max_snap.get('timestamp', 'Unknown')})")
    
    # Setup NetworkX Graph
    G = nx.DiGraph()
    
    edge_dict = {}
    if isinstance(max_snap, dict):
        edge_dict = max_snap.get('edge_index_dict', max_snap.get('edges', {}))
    else:
        # PyG HeteroData
        edge_dict = max_snap.edge_index_dict
    
    node_colors = []
    colors_map = {
        "Patient": "#1f77b4",
        "Practitioner": "#2ca02c",
        "Organization": "#d62728",
        "Encounter": "#ff7f0e",
        "Condition": "#9467bd",
        "Observation": "#17becf",
    }

    for (src_type, rel, dst_type), (src_idxs, dst_idxs) in edge_dict.items():
        for s, d in zip(src_idxs, dst_idxs):
            u = f"{src_type}_{s}"
            v = f"{dst_type}_{d}"
            G.add_node(u, color=colors_map.get(src_type, "grey"))
            G.add_node(v, color=colors_map.get(dst_type, "grey"))
            G.add_edge(u, v)
            
    # Draw
    plt.figure(figsize=(10, 8))
    pos = nx.spring_layout(G, k=0.6, seed=42)
    
    colors = [G.nodes[n].get('color', 'grey') for n in G.nodes()]
    nx.draw(G, pos, node_color=colors, with_labels=False, node_size=300, alpha=0.9)
    
    # Legend
    from matplotlib.lines import Line2D
    legend_elements = [Line2D([0], [0], marker='o', color='w', label=k, markerfacecolor=v, markersize=10) 
                       for k, v in colors_map.items() if any(k in n for n in G.nodes())]
    plt.legend(handles=legend_elements, loc='upper right')
    plt.title(f"Interactions on {str(max_snap.get('timestamp', ''))[:10]}")
    plt.show()
            
else:
    print("No snapshots generated.")

## 4. Hyperbolic Embeddings
Learn hierarchical representations of medical concepts using Poincaré embeddings. This places general concepts near the origin and specific concepts near the boundary of the hyperbolic ball.

In [ ]:
from neurofhir.hyperbolic_encoder import PoincareEmbedding
import torch

# 1. Define a dummy ontology (e.g., SNOMED subset)
ontology_map = {
    "Disorder": ["Respiratory", "Cardiovascular"],
    "Respiratory": ["Asthma", "Pneumonia"],
    "Cardiovascular": ["Hypertension", "Arrhythmia"]
}

# Map codes to indices
codes = ["Disorder", "Respiratory", "Cardiovascular", "Asthma", "Pneumonia", "Hypertension", "Arrhythmia"]
idx_to_code = {i: code for i, code in enumerate(codes)}

# 2. Initialize Model
model = PoincareEmbedding(
    num_embeddings=len(idx_to_code),
    embedding_dim=2, # Use 2D for easy visualization
    ontology_map=ontology_map,
    idx_to_code=idx_to_code
)

# 3. Compute Embeddings (Untrained random initialization for demo)
# In a real loop, you would optimize the loss to minimize distance between parents/children.
ids = torch.arange(len(idx_to_code))
vectors = model(ids).detach()

# 4. Visualize in Poincaré Disk
plt.figure(figsize=(6, 6))
plt.xlim(-1.1, 1.1)
plt.ylim(-1.1, 1.1)
# Draw unit circle
circle = plt.Circle((0, 0), 1, color='black', fill=False)
plt.gca().add_artist(circle)

for i, vec in enumerate(vectors):
    plt.scatter(vec[0], vec[1], label=idx_to_code[i])
    plt.text(vec[0]+0.02, vec[1]+0.02, idx_to_code[i])

plt.title("Concepts in Hyperbolic Space (Random Init)")
plt.grid(True, linestyle='--')
plt.show()

## 5. Causal Edge Mining
Discover potential causal links (e.g., Medication -> Condition Change) from the patient history.

In [ ]:
from neurofhir.causal_edge_miner import CausalEdgeMiner

miner = CausalEdgeMiner()

# Mine relationships from the loaded patient resources
cleaned_edges = miner.mine_relationships(resources)

if cleaned_edges is not None and not cleaned_edges.is_empty():
    print("Found potential causal edges:")
    # Show top edges sorted by weight (confidence)
    print(cleaned_edges.select(["source", "relation", "target", "weight"]).head(5))
else:
    print("No strong causal patterns found in this patient data.")